# The functools module

## Introduction to functools

Python comes with a fun module called functools. The functions inside functools are considered “higher-order” functions which can act on or return other functions. 

## Caching with functools.lru_cache

A decorator that adds caching to the function it decorates.

In [1]:
import urllib.error
import urllib.request

from functools import lru_cache


@lru_cache(maxsize=24)
def get_webpage(module):
    """
    Gets the specified Python module web page
    """    
    webpage = "https://docs.python.org/3/library/{}.html".format(module)
    try:
        with urllib.request.urlopen(webpage) as request:
            return request.read()
    except urllib.error.HTTPError:
        return None

if __name__ == '__main__':
    modules = ['functools', 'collections', 'os', 'sys']
    for module in modules:
        page = get_webpage(module)
        if page:
            print("{} module page found".format(module))

functools module page found
collections module page found
os module page found
sys module page found


## functool.partial

One of the functools classes is the partial class. You can use it create a new function with partial application of the arguments and keywords that you pass to it. You can use partial to “freeze” a portion of your function’s arguments and/or keywords which results in a new object.

In [2]:
# feeding arguement one input at a time

from functools import partial

def add(x, y):
    return x + y

p_add = partial(add, 3)
print(p_add(8))
print(p_add(2))
print(p_add(1))
print(p_add(5))

11
5
4
8


In [3]:
# Using partial to set action and dynamically pass label of button clicked in GUI

import wx

from functools import partial 


class MainFrame(wx.Frame):
    """
    This app shows a group of buttons
    """

    def __init__(self, *args, **kwargs):
        """Constructor"""
        super(MainFrame, self).__init__(parent=None, title='Partial')
        panel = wx.Panel(self)

        sizer = wx.BoxSizer(wx.VERTICAL)
        btn_labels = ['one', 'two', 'three']
        for label in btn_labels:
            btn = wx.Button(panel, label=label)
            btn.Bind(wx.EVT_BUTTON, partial(self.onButton, label=label))
            sizer.Add(btn, 0, wx.ALL, 5)

        panel.SetSizer(sizer)
        self.Show()

    def onButton(self, event, label):
        """
        Event handler called when a button is pressed
        """
        print('You pressed: ' +  str(label))


if __name__ == '__main__':
    app = wx.App(False)
    frame = MainFrame()
    app.MainLoop() 

You pressed: one
You pressed: two
You pressed: three


In [4]:
# Another application of partial

from functools import partial


def add(x, y):
    """"""
    return x + y


def multiply(x, y):
    """"""
    return x * y


def run(func):
    """"""
    print(func())


def main():
    """"""
    a1 = partial(add, 1, 2)
    m1 = partial(multiply, 5, 8)
    run(a1)
    run(m1)
    

if __name__ == "__main__":
    main()

3
40


## Function Overloading with functiils.singledispatch

This decorator will transform your regular function into a single dispatch generic function. 

In [11]:
from functools import singledispatch


@singledispatch
def add(a, b):
    raise NotImplementedError('Unsupported type')


@add.register(int)
def _(a, b):
    print("First argument is of type ", type(a))
    print(a + b)


@add.register(str)
def _(a, b):
    print("First argument is of type ", type(a))
    print(a + b)


@add.register(list)
def _(a, b):
    print("First argument is of type ", type(a))
    print(a + b)


if __name__ == '__main__':
    add(1, 2)
    add('Python', 'Programming')
#     add([1, 2, 3], [5, 6, 7])
    print('\n')
    
# See supported data types
print(add.registry.keys())

First argument is of type  <class 'int'>
3
First argument is of type  <class 'str'>
PythonProgramming


dict_keys([<class 'object'>, <class 'int'>, <class 'str'>, <class 'list'>])


In [27]:
# Decorating stacking to add support for multiple data types at once

from functools import singledispatch
from decimal import Decimal


@singledispatch
def add(a, b):
    raise NotImplementedError('Unsupported type')

@add.register(float)
@add.register(Decimal)
@add.register(str)
def _(a, b):
    print("First argument is of type ", type(a))
    print(a + b)


if __name__ == '__main__':
    add(1.23, 5.5)
    print('\n')
    add(Decimal(100.5), Decimal(10.789))
    print('\n')
    add('checking overloaded', ' function\'s support for strings!')

First argument is of type  <class 'float'>
6.73


First argument is of type  <class 'decimal.Decimal'>
111.2889999999999997015720510


First argument is of type  <class 'str'>
checking overloaded function's support for strings!


## An issue with using decorators

You can use wraps as a decorator to fix docstrings and names of decorated functions. Why does this matter? This sounds like a weird edge case at first, but if you’re writing an API or any code that someone other than yourself will be using, then this could be important. The reason being that when you use Python’s introspection to figure out someone else’s code, a decorated function will return the wrong information. Let’s look at a simple example that I have dubbed decorum.py:

In [31]:
# Example of decorated functino returning wrong info
# Basically what is happening here is that the decorator is 
# changing the decorated function’s name and docstring to its own.

# decorum.py


def another_function(func):
    """
    A function that accepts another function
    """

    def wrapper():
        """
        A wrapping function
        """
        val = "The result of %s is %s" % (func(),
                                          eval(func())
                                          )
        return val
    return wrapper


@another_function
def a_function():
    """A pretty useless function"""
    return "1+1"


if __name__ == "__main__":
    print(a_function.__name__)
    print(a_function.__doc__)

wrapper

        A wrapping function
        


## functools.wraps to the Rescue!

Here we wrap out wrapper with a wraps() decorator and it fixe the issue.

In [35]:
from functools import wraps


def another_function(func):
    """
    A function that accepts another function
    """

    @wraps(func)
    def wrapper():
        """
        A wrapping function
        """
        val = "The result of %s is %s" % (func(),
                                          eval(func())
                                          )
        return val
    return wrapper


@another_function
def a_function():
    """A pretty useless function"""
    return "1+1"


if __name__ == "__main__":
    print(a_function.__name__)
    print(a_function.__doc__)

a_function
A pretty useless function
